In [47]:
# Porcentagem ou exclamação é uma "magic command" do Google Colab,
# usada para instalar pacotes diretamente no ambiente do notebook
%pip -q install google-genai


In [48]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [49]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [50]:
# Importa "HTML e Markdown"

from IPython.display import HTML, Markdown

In [51]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Qual é a empresa responsavel por desenvolver o Gemini?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 O Gemini é desenvolvido pelo Google, mais especificamente pela Google DeepMind, uma subsidiária da Google especializada em Inteligência Artificial e Machine Learning.


In [52]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['qual empresa desenvolveu Gemini', 'empresa desenvolvedora do Gemini']
Páginas utilizadas na resposta: ipnet.cloud



In [53]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [54]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [55]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [56]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [57]:
##########################################
# --- Agente 1: Buscador de Jogos --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model=MODEL_ID,
        description="Agente que busca lista de jogos eletrônicos",
        tools=[google_search],
        instruction="""
          Você é um agente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search) para buscar uma lista com os melhores jogos do ano anterior.
          Foque em no máximo 5 melhores jogos, separados por categorias.
          Exclua jogos que possuem temas eróticos.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [58]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscador, você deve:
          usar a ferramenta de busca do Google (google_search) para criar um plano sobre quais são os jogos mais populares do ano anterior, você pode usar o (google_search) para encontrar a lista mais relevante com base nas suas pesquisas e retornar esse tema.
        """,
        description="Agente que planeja posts sobre a lista de jogos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [59]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODEL_ID,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você vai escrever um resumo sobre os jogos mais populares do ano anterior.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para um website sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final para compartilhamentos em redes sociais.
            """,
        description="Agente redator de posts engajadores para website"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [60]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts sobre jogos eletrônicos, com foco em websites e blogs.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de website e blog abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para websites e blogs."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [61]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Websites e Blogs com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite qual CATEGORIA DE JOGOS quer criar o post. Será gerado um TOP 5 para o website ou blog: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar a categoria!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)

  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Websites e Blogs com 4 Agentes 🚀
❓ Por favor, digite qual CATEGORIA DE JOGOS quer criar o post. Será gerado um TOP 5 para o website ou blog: Corrida
Maravilha! Vamos então criar o post sobre novidades em Corrida

--- Resultado do Agente 1 (Buscador) ---



> Ok, aqui estão algumas buscas que posso fazer para encontrar os melhores jogos de corrida de 2024:
> 
>  
> Com base nas minhas buscas, aqui estão alguns dos melhores jogos de corrida de 2024, separados por categoria:
> 
>  **Simuladores de Corrida:**
> 
>  *   **Gran Turismo 7:** Apresenta gráficos ultrarrealistas, grande variedade de carros e pistas e um modo online bem desenvolvido.
>  *   **Assetto Corsa Competizione:** Conhecido pela sua física realista e ótima experiência em VR.
> 
>  **Corrida Arcade:**
> 
>  *   **Hot Wheels Unleashed 2:** Oferece um extenso leque de veículos e permite criar as suas próprias pistas.
>  *   **Mario Kart 8 Deluxe:** Suporta até quatro jogadores na mesma prova e permite jogar pequenos campeonatos.
> 
>  **Mundo Aberto:**
> 
>  *   **Forza Horizon 5:**  Oferece uma enorme variedade de carros e conteúdo extenso.
> 
> Além destes, também foram mencionados outros jogos como F1 23, Need for Speed Unbound, Dirt 5, WRC Generations e jogos gratuitos na Steam como Trackmania, Disney Speedstorm e Asphalt Legends Unite.
> 


--------------------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Com certeza! Para criar um plano de conteúdo atraente sobre os jogos de corrida mais populares de 2024, podemos abordar os seguintes temas e formatos:
> 
> **1. Lista dos Melhores Jogos de Corrida de 2024 por Categoria:**
> 
> *   **Post de Blog/Artigo:** "Os Melhores Jogos de Corrida de 2024: Simuladores, Arcade e Mundo Aberto".
>     *   **Conteúdo:** Detalhar cada jogo listado (Gran Turismo 7, Assetto Corsa Competizione, Hot Wheels Unleashed 2, Mario Kart 8 Deluxe, Forza Horizon 5, etc.), mencionando seus pontos fortes, características únicas e para qual tipo de jogador ele é mais indicado.
>     *   **SEO:** Palavras-chave como "melhores jogos de corrida 2024", "simuladores de corrida", "jogos de corrida arcade", "jogos de mundo aberto", "Forza Horizon 5", "Gran Turismo 7", etc.
> *   **Vídeo no YouTube:** "Top 5 Jogos de Corrida Imperdíveis de 2024".
>     *   **Conteúdo:** Mostrar gameplays de cada jogo, com comentários sobre os gráficos, jogabilidade e o que torna cada um especial.
> *   **Infográfico:** "Guia Rápido dos Melhores Jogos de Corrida de 2024".
>     *   **Conteúdo:** Comparar os jogos visualmente, com ícones para indicar características como "Realismo", "Diversão Arcade", "Mundo Aberto", "Multiplayer Local", etc.
> 
> **2. Destaque para Simuladores de Corrida:**
> 
> *   **Post no Instagram:** "Para os Amantes do Realismo: Gran Turismo 7 e Assetto Corsa Competizione".
>     *   **Conteúdo:** Fotos e vídeos dos jogos, com legendas destacando a física realista, gráficos impressionantes e a experiência imersiva de VR.
> *   **Review Detalhada no Blog:** "Assetto Corsa Competizione: O Simulador Definitivo para Fãs de GT".
>     *   **Conteúdo:** Análise aprofundada da física, dos carros, das pistas e da experiência online.
> 
> **3. Foco em Corrida Arcade e Diversão:**
> 
> *   **TikTok:** Vídeos curtos e divertidos de Hot Wheels Unleashed 2 e Mario Kart 8 Deluxe.
>     *   **Conteúdo:** Momentos engraçados, manobras radicais e a jogabilidade caótica e divertida desses jogos.
> *   **Enquete no Twitter:** "Qual o Melhor Jogo de Corrida Arcade de 2024?".
>     *   **Opções:** Hot Wheels Unleashed 2, Mario Kart 8 Deluxe, Disney Speedstorm, Asphalt Legends Unite.
> 
> **4. Mundo Aberto e Exploração:**
> 
> *   **Post no Facebook:** "Forza Horizon 5: A Combinação Perfeita de Corrida e Mundo Aberto".
>     *   **Conteúdo:** Fotos e vídeos mostrando a beleza dos cenários, a variedade de carros e as atividades disponíveis no jogo.
> *   **Artigo no Blog:** "Forza Horizon 5: Um Guia para Iniciantes no Mundo Aberto das Corridas".
>     *   **Conteúdo:** Dicas para explorar o mapa, participar de eventos e personalizar carros.
> 
> **5. Outros Jogos e Destaques:**
> 
> *   **Post no LinkedIn:** "F1 23: A Experiência Definitiva da Fórmula 1 nos Games".
>     *   **Conteúdo:** Focar na simulação precisa da F1, nos gráficos realistas e na experiência de carreira.
> *   **Twitch:** Transmissões ao vivo jogando Trackmania, Disney Speedstorm e Asphalt Legends Unite.
>     *   **Conteúdo:** Interagir com o público, mostrar as mecânicas únicas de cada jogo e fazer desafios.
> 
> **Hashtags:**
> 
> *   \#jogosdecorrida \#corrida \#gaming \#videogames \#simuladordecorrida \#arcadegaming \#mundogamer \#forzahorizon5 \#granturismo7 \#mariokart \#hotwheelsunleashed \#f123 \#trackmania \#disneyspeedstorm \#asphaltlegends
> 
> **Observações:**
> 
> *   Adapte o conteúdo para cada plataforma, utilizando os formatos e estilos mais adequados.
> *   Use imagens e vídeos de alta qualidade para tornar os posts mais atraentes.
> *   Interaja com o público, respondendo a comentários e perguntas.
> *   Monitore as tendências e novidades do mundo dos jogos de corrida para manter o conteúdo sempre atualizado.
> 
> Com este plano, você poderá criar conteúdo diversificado e interessante para atrair fãs de corrida de todas as plataformas!
> 


--------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> **Prepare-se para a Aceleração: Os Jogos de Corrida Mais Insanos de 2024!** 
> 
> 2024 foi um ano ÉPICO para os amantes da velocidade virtual! 
> 
> Tivemos desde simuladores realistas que te fazem sentir cada curva, até jogos arcade que desafiam a gravidade com manobras INSANAS. 
> 
> Se liga nos destaques:
> 
> *   **Gran Turismo 7:** Para os apaixonados por realismo, GT7 elevou o nível com gráficos de cair o queixo e física impecável.
> *   **Hot Wheels Unleashed 2:** A nostalgia e a diversão arcade se encontraram nesse jogo frenético, perfeito para quem quer zerar no role play.
> *   **Forza Horizon 5:** Explorar um mundo aberto gigante enquanto pilota carrões alucinantes? Forza Horizon 5 entregou tudo!
> 
> E aí, qual foi seu jogo de corrida favorito de 2024? Conta pra gente nos comentários! 
> 
> \#jogosdecorrida \#corrida \#gaming \#videogames
> 


--------------------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais interessante e otimizado para o público jovem! Aqui estão algumas sugestões:
> 
> *   **Título:** O título atual é bom, mas podemos adicionar um toque de mistério ou exclusividade. Que tal algo como "Acelerando em 2024: Os 5 Jogos de Corrida Que Você PRECISA Jogar!"? Listas numeradas costumam atrair mais cliques.
> *   **Introdução:** A introdução está um pouco genérica. Que tal algo mais chamativo e direto ao ponto? Exemplo: "2024 já está na história para os fãs de jogos de corrida! De simuladores ultra realistas a arcades insanos, prepare-se para conhecer os jogos que botaram fogo nas pistas (virtuais, claro!)."
> *   **Listagem dos jogos:**
>     *   **Gran Turismo 7:** Ótimo para quem busca realismo, mas podemos adicionar um detalhe extra: "Se você é do tipo que passa horas ajustando a pressão dos pneus, GT7 é seu paraíso. Gráficos de nova geração e física que te fazem sentir cada grama de força G."
>     *   **Hot Wheels Unleashed 2:** Boa descrição, mas podemos adicionar um pouco mais de hype: "Aquele gostinho de infância turbinado! Hot Wheels Unleashed 2 é pura adrenalina arcade. Manobras insanas, pistas de tirar o fôlego e a nostalgia que a gente ama."
>     *   **Forza Horizon 5:** Excelente para quem curte mundo aberto, mas podemos ser mais específicos: "Imagine dirigir um supercarro em paisagens de tirar o fôlego... Forza Horizon 5 te dá essa liberdade! Explore um mapa GIGANTE, participe de eventos insanos e mostre quem manda no asfalto (ou na terra, na neve...).
> *   **Chamada para ação (CTA):** A pergunta no final é boa, mas podemos ser mais incisivos: "E aí, qual desses jogos te fez acelerar em 2024? Deixe seu comentário e compartilhe sua experiência! E não esqueça de marcar aquele amigo(a) que também ama jogos de corrida!"
> *   **Hashtags:** As hashtags estão ok, mas podemos adicionar algumas mais específicas e populares: \#GT7 \#HotWheelsUnleashed2 \#ForzaHorizon5 \#SimRacing \#ArcadeRacing \#Gamer \#GamingLife
> *   **Tom:** O tom geral está bom, mas podemos adicionar um pouco mais de empolgação e usar mais gírias/expressões que o público jovem usa.
> 
> Com essas pequenas mudanças, o post ficará ainda mais atraente e engajador!
> 


--------------------------------------------------------------
